In [169]:
import rpy2.robjects as ro
import pandas as pd
import json
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

r = ro.r
r['source']('ParseNFLPlayers.R')

player_stats_r = ro.globalenv['player_stats']
team_data_r = ro.globalenv['team_data']
player_stats_kicker_r = ro.globalenv['player_stats_kicker']
roster_r = ro.globalenv['roster']
official_player_stats_r = ro.globalenv['official_player_stats']
nextgen_stats_r = ro.globalenv['nextgen_stats']
combine_r = ro.globalenv['combine']

# Convert the tibble to a pandas data frame
with localconverter(ro.default_converter + pandas2ri.converter):
    df_player_stats = ro.conversion.rpy2py(player_stats_r)
    df_team_data = ro.conversion.rpy2py(team_data_r)
    df_player_stats_kicker = ro.conversion.rpy2py(player_stats_kicker_r)
    df_roster = ro.conversion.rpy2py(roster_r)
    df_official_player_stats = ro.conversion.rpy2py(official_player_stats_r)
    df_nextgen_stats = ro.conversion.rpy2py(nextgen_stats_r)
    df_combine = ro.conversion.rpy2py(combine_r)
    
df_player_stats.head()

,player_id,player_name,recent_team,season,week,season_type,completions,attempts,passing_yards,passing_tds,...,receiving_epa,receiving_2pt_conversions,special_teams_tds,fantasy_points,fantasy_points_ppr,pacr,racr,target_share,air_yards_share,wopr
1,00-0019596,T.Brady,TB,2021,1,REG,32,50,379.0,4,...,NaN,0,0.0,27.16,27.16,0.849776,NaN,NaN,NaN,NaN
2,00-0019596,T.Brady,TB,2021,2,REG,24,36,276.0,5,...,NaN,0,0.0,29.64,29.64,0.795389,NaN,NaN,NaN,NaN
3,00-0019596,T.Brady,TB,2021,3,REG,41,55,432.0,1,...,NaN,0,0.0,28.68,28.68,1.130890,NaN,NaN,NaN,NaN
4,00-0019596,T.Brady,TB,2021,4,REG,22,43,269.0,0,...,NaN,0,0.0,11.06,11.06,0.586057,NaN,NaN,NaN,NaN
5,00-0019596,T.Brady,TB,2021,5,REG,30,41,411.0,5,...,NaN,0,0.0,37.74,37.74,0.976247,NaN,NaN,NaN,NaN


In [115]:
name = "Tom Brady"
name = name.lower()
tom_df = df_roster[df_roster['full_name'].str.lower() == name]
del tom_df['season']
tom_df = tom_df.head(1)

In [116]:
player_name_str = str(tom_df["first_name"][0][0:1] + "." + tom_df["last_name"][0])

player_name_str
df_team_data['team_abbr']
tom_stats_factor['team']
tom_df




,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,...,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url
2510,TB,QB,QB,12,Active,Tom Brady,Tom,Brady,2771.0,76,...,2330,41c44740-d0f6-44ab-8347-3b5d515e5ecf,5228,1350,698,BradTo00,4314,167,22,https://static.www.nfl.com/image/private/f_aut...


In [238]:
tom_stats = df_player_stats[df_player_stats['player_name'] == player_name_str]
# tom_stats = tom_stats[tom_stats['recent_team'][0] == tom_df['team'][0]]
tom_stats_numeric = tom_df.select_dtypes(include='number')
tom_stats_factor = tom_df.select_dtypes(exclude='number')
tom_team = df_team_data[df_team_data['team_abbr'] == tom_stats_factor['team'][0]]

tom_df.columns




Index(['team', 'position', 'depth_chart_position', 'jersey_number', 'status',
       'full_name', 'first_name', 'last_name', 'birth_date', 'height',
       'weight', 'college', 'high_school', 'gsis_id', 'espn_id',
       'sportradar_id', 'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id',
       'fantasy_data_id', 'sleeper_id', 'years_exp', 'headshot_url'],
      dtype='object')

In [248]:
tom_df['index'] = 
tom_stats_numeric['index'] = 1
tom_df.set_index('index')
tom_stats_numeric.set_index('index')


tom_final = pd.concat([tom_df, tom_stats_numeric, tom_stats_factor], axis=1, join="inner")
tom_team = tom_team.rename(columns={'team_abbr' : 'team'})
tom_final = tom_final.loc[:,~tom_final.columns.duplicated()]
tom_final = pd.merge(tom_final, tom_team)
tom_final= pd.merge(tom_final, tom_stats_factor)

tom_stats_numeric





<ipython-input-248-676914a0b019>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tom_stats_numeric['index'] = 1


,jersey_number,birth_date,espn_id,yahoo_id,rotowire_id,pff_id,fantasy_data_id,years_exp,index
2510,12,2771.0,2330,5228,1350,698,4314,22,1


In [160]:
tom_final = tom_final.loc[:,~tom_final.columns.duplicated()]

tom_final.to_json(orient="records")




'[{"team":"TB","position":"QB","depth_chart_position":"QB","jersey_number":12,"status":"Active","full_name":"Tom Brady","first_name":"Tom","last_name":"Brady","birth_date":2771.0,"height":"76","weight":"225","college":"Michigan","high_school":"Junipero Serra (CA)","gsis_id":"00-0019596","espn_id":2330,"sportradar_id":"41c44740-d0f6-44ab-8347-3b5d515e5ecf","yahoo_id":5228,"rotowire_id":1350,"pff_id":698,"pfr_id":"BradTo00","fantasy_data_id":4314,"sleeper_id":"167","years_exp":22,"headshot_url":"https:\\/\\/static.www.nfl.com\\/image\\/private\\/f_auto,q_auto\\/league\\/n7fj5fjblf6pdgjdtphr"}]'

In [234]:
def get_player_data(player_name_query):
    player_name_query = player_name_query.lower()
    filtered_player_roster = df_roster[df_roster['full_name'].str.lower() == player_name_query]
    filtered_player_roster = filtered_player_roster.head(1)
    player_name_str = str(filtered_player_roster["first_name"][0][0:1] + "." + filtered_player_roster["last_name"][0])
    player_stats_raw = df_player_stats[df_player_stats['player_name'] == player_name_str]
    player_numeric_data = filtered_player_roster.select_dtypes(include='number')
    player_factor_data = filtered_player_roster.select_dtypes(exclude='number')
    player_team_data = df_team_data[df_team_data['team_abbr'] == player_factor_data['team'][0]]
    player_team_data.rename(columns={'team_abbr' : 'team'})
    full_player_data = pd.concat([filtered_player_roster, player_numeric_data, player_factor_data], axis=1, join="inner")
    player_team_data = player_team_data.rename(columns={'team_abbr' : 'team'})
    full_player_data = full_player_data.loc[:,~full_player_data.columns.duplicated()]
    full_player_data = pd.merge(full_player_data, player_team_data)
    player_json = full_player_data.to_json(orient="records")
    return player_json







    
    
    

In [235]:
get_player_data("Patrick Mahomes")



'[{"season":2021.0,"team":"KC","position":"QB","depth_chart_position":"QB","jersey_number":15,"status":"Active","full_name":"Patrick Mahomes","first_name":"Patrick","last_name":"Mahomes","birth_date":9390.0,"height":"75","weight":"227","college":"Texas Tech","high_school":"Whitehouse (TX)","gsis_id":"00-0033873","espn_id":3139477,"sportradar_id":"11cad59d-90dd-449c-a839-dddaba4fe16c","yahoo_id":30123,"rotowire_id":11839,"pff_id":11765,"pfr_id":"MahoPa00","fantasy_data_id":18890,"sleeper_id":"4046","years_exp":5,"headshot_url":"https:\\/\\/static.www.nfl.com\\/image\\/private\\/f_auto,q_auto\\/league\\/tlg2oo5yrk6mins3nxxe","team_name":"Kansas City Chiefs","team_id":"2310","team_nick":"Chiefs","team_conf":"AFC","team_division":"AFC West","team_color":"#E31837","team_color2":"#FFB612","team_color3":"#000000","team_color4":"#e31837","team_logo_wikipedia":"https:\\/\\/upload.wikimedia.org\\/wikipedia\\/en\\/thumb\\/e\\/e1\\/Kansas_City_Chiefs_logo.svg\\/100px-Kansas_City_Chiefs_logo.svg.pn